# Goal 3: Calibration and Rectification

### Step #01: Load images 

In [5]:
import numpy as np
import cv2
import glob
import os
import datetime

In [6]:
# ---------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------
RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

In [7]:
# Board definitions
boards = [
    {"name": "boardBig", "pattern_size": (11,7)},
    {"name": "boardMed", "pattern_size": (5,7)},
]

# VISUALIZATION SETTINGS
VISUALIZE = True
VIS_DELAY_MS = 0  # Wait 500ms between images. Set to 0 to wait for keypress.

# !!! CRITICAL !!! 
# Measure your real square size. 
# USE METERS to match KITTI format (e.g., 30mm = 0.03)
REAL_SQUARE_SIZE_M = 0.1

# Precompute object point templates
obj_templates = {}
for b in boards:
    nx, ny = b["pattern_size"]
    # We apply the real world scale here
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) * REAL_SQUARE_SIZE_M
    obj_templates[b["name"]] = objp

In [8]:
import cv2
import numpy as np
import glob
import os

# ---------------------------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------------------------

RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

# Board definitions (Physical: 12x8 squares -> 11x7 corners; 6x8 squares -> 5x7 corners)
boards = [
    {"name": "boardBig", "pattern_size": (11, 7)},
    {"name": "boardMed", "pattern_size": (5, 7)},
]

VISUALIZE = True
VIS_DELAY_MS = 0  # 0 = Wait for keypress. Change to 1 or 500 for auto-advance.

# ---------------------------------------------------------------------------
# 2. IMAGE PROCESSING & DETECTION LOGIC
# ---------------------------------------------------------------------------

def get_image_files(directory):
    if not os.path.exists(directory):
        print(f"[ERROR] Directory does not exist: {directory}")
        return []
    files = sorted(glob.glob(os.path.join(directory, "*.png")))
    if not files:
        files = sorted(glob.glob(os.path.join(directory, "*.jpg")))
    return files

def sharpen_image(gray_img):
    """
    Simple sharpening kernel to pop edges.
    """
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(gray_img, -1, kernel)

def find_all_boards_multiscale(img, display_prefix=""):
    """
    Robustly finds boards using:
    1. Sharpening
    2. Masking (for multiple instances)
    3. UPSCALING (for distant/small boards)
    """
    vis_img = img.copy()
    
    # Prepare Base Images
    gray_base = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Processing copy (Sharpened + used for masking)
    gray_processing = sharpen_image(gray_base)

    all_detections = []
    
    # Subpixel criteria
    subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 0.001)
    
    total_found = 0

    for board in boards:
        name = board["name"]
        nx, ny = board["pattern_size"]
        
        instance_count = 0
        
        while True:
            # --- ATTEMPT 1: Normal Scale ---
            flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
            ret, corners = cv2.findChessboardCorners(gray_processing, (nx, ny), flags)
            
            scale_used = 1.0

            # --- ATTEMPT 2: Upscale (2x Zoom) ---
            if not ret:
                gray_upscaled = cv2.resize(gray_processing, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
                ret, corners_up = cv2.findChessboardCorners(gray_upscaled, (nx, ny), flags)
                
                if ret:
                    corners = corners_up / 2.0
                    scale_used = 2.0

            # --- PROCESS RESULT ---
            if ret:
                instance_count += 1
                total_found += 1
                
                # Refine Subpixels (Always on clean original base)
                corners_refined = cv2.cornerSubPix(gray_base, corners, (11, 11), (-1, -1), subpix_criteria)
                
                all_detections.append({
                    "name": name, 
                    "instance": instance_count,
                    "corners": corners_refined
                })

                # Visualization
                cv2.drawChessboardCorners(vis_img, (nx, ny), corners_refined, ret)
                
                # Label
                lbl = f"{name}#{instance_count}" + ("(Z)" if scale_used > 1 else "")
                label_pos = tuple(corners_refined[0].ravel().astype(int) - [5, 5])
                cv2.putText(vis_img, lbl, label_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

                # MASKING
                corners_int = corners.astype(np.int32)
                hull = cv2.convexHull(corners_int)
                
                mask = np.zeros_like(gray_processing)
                cv2.fillConvexPoly(mask, hull, 255)
                
                # Dilate mask
                kernel_dilate = np.ones((15, 15), np.uint8)
                mask_dilated = cv2.dilate(mask, kernel_dilate)
                
                # Apply mask
                gray_processing[mask_dilated > 0] = 0
                
                # print(f"[{display_prefix}] Found {name} #{instance_count}")
            else:
                break

    # Summary Label
    cv2.putText(vis_img, f"{display_prefix} Total: {total_found}", (30, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    return vis_img, all_detections

# ---------------------------------------------------------------------------
# 3. MAIN EXECUTION
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    left_files = get_image_files(LEFT_IMG_PATH)
    right_files = get_image_files(RIGHT_IMG_PATH)

    if not left_files or not right_files:
        print("Error: Could not find images in provided paths.")
        exit()

    # Ensure we process matching pairs
    num_pairs = min(len(left_files), len(right_files))
    print(f"Found {num_pairs} stereo pairs. Starting processing...")
    print("Controls: Press any key to next image, 'ESC' to quit.")

    for i in range(num_pairs):
        f_left = left_files[i]
        f_right = right_files[i]
        filename = os.path.basename(f_left)

        print(f"\n[{i+1}/{num_pairs}] Processing: {filename}")

        imgL = cv2.imread(f_left)
        imgR = cv2.imread(f_right)

        if imgL is None or imgR is None:
            print("  Skipping - Read Error")
            continue
        
        # Detect
        visL, detsL = find_all_boards_multiscale(imgL, "L")
        visR, detsR = find_all_boards_multiscale(imgR, "R")

        # Summary Print
        print(f"  Left:  {len(detsL)} boards")
        print(f"  Right: {len(detsR)} boards")
        
        if len(detsL) != len(detsR):
            print("  [!] Mismatch detected.")

        # Visualization
        if VISUALIZE:
            combined_vis = np.hstack((visL, visR))
            
            # Add file info to the visual
            cv2.putText(combined_vis, f"Frame: {filename}", (10, combined_vis.shape[0] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # Resize for easier viewing on screen (0.6 scale)
            scale = 0.6
            combined_vis_resized = cv2.resize(combined_vis, None, fx=scale, fy=scale)
            
            cv2.imshow('Stereo Board Detection', combined_vis_resized)
            
            # Wait Logic
            key = cv2.waitKey(VIS_DELAY_MS)
            if key == 27: # ESC key
                print("Stopping execution by user.")
                break
    
    cv2.destroyAllWindows()
    print("Done.")

Found 19 stereo pairs. Starting processing...
Controls: Press any key to next image, 'ESC' to quit.

[1/19] Processing: 0000000000.png
  Left:  12 boards
  Right: 12 boards

[2/19] Processing: 0000000001.png
  Left:  12 boards
  Right: 12 boards

[3/19] Processing: 0000000002.png
  Left:  12 boards
  Right: 12 boards

[4/19] Processing: 0000000003.png
  Left:  12 boards
  Right: 12 boards

[5/19] Processing: 0000000004.png
  Left:  12 boards
  Right: 12 boards

[6/19] Processing: 0000000005.png
  Left:  12 boards
  Right: 12 boards

[7/19] Processing: 0000000006.png
  Left:  12 boards
  Right: 12 boards

[8/19] Processing: 0000000007.png
  Left:  12 boards
  Right: 12 boards

[9/19] Processing: 0000000008.png
  Left:  12 boards
  Right: 12 boards

[10/19] Processing: 0000000009.png
  Left:  12 boards
  Right: 12 boards

[11/19] Processing: 0000000010.png
  Left:  12 boards
  Right: 12 boards

[12/19] Processing: 0000000011.png
  Left:  12 boards
  Right: 12 boards

[13/19] Processing: 